[문서 업로드] → [문서 로딩/분할] → [임베딩/벡터DB 저장] → [검색/질의] → [LLM 요약/분석] → [결과 제공]

In [24]:
import os
import re
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from typing import List, Dict, Any


In [ ]:
def load_raw_papers_from_pdfs(paper_dir: str) -> List[Document]:
    """
    지정된 디렉토리에서 PDF 파일을 로드하여 원본 Document 객체 리스트를 반환합니다.
    각 Document는 PDF의 한 페이지에 해당하며, 원본 텍스트와 메타데이터를 포함합니다.
    """
    pdf_files = [os.path.join(paper_dir, f) for f in os.listdir(paper_dir) if f.endswith('.pdf')]
    
    all_raw_pages: List[Document] = []
    if not pdf_files:
        print(f"No PDF files found in '{paper_dir}'.")
        return all_raw_pages

    for pdf_file in pdf_files:
        print(f"Loading (raw): {os.path.basename(pdf_file)}")
        loader = PyPDFLoader(pdf_file)
        try:
            # load()는 페이지별 Document 객체 리스트를 반환
            pages_from_pdf = loader.load()
            all_raw_pages.extend(pages_from_pdf)
        except Exception as e:
            print(f"Error loading {pdf_file}: {e}")
            continue
            
    print(f"\nTotal raw pages loaded: {len(all_raw_pages)}")
    return all_raw_pages
def preprocess_page_content(text: str, source_filename: str = "", page_num: int = -1) -> str:
    """
    개별 페이지 내용에서 불필요한 요소를 제거하고 텍스트를 정제합니다.
    논문 형식에 맞춰 정규표현식을 정교하게 조정해야 합니다.
    """
    # 이전 답변과 동일한 전처리 로직
    text = re.sub(r"^\s*The NEW ENGLAND JOURNAL of MEDICINE\s*\n", "", text, flags=re.IGNORECASE | re.MULTILINE)
    text = re.sub(r"^\s*N Engl J Med\s+\d{4};\d+:\d+-\d+\s*\n", "", text, flags=re.IGNORECASE | re.MULTILINE)
    text = re.sub(r"^\s*VOL\.\s*\d+\s*NO\.\s*\d+\s*\n", "", text, flags=re.IGNORECASE | re.MULTILINE)
    text = re.sub(r"^\s*NEJM\.ORG\s*JUNE\s*\d{1,2},\s*\d{4}\s*\n", "", text, flags=re.IGNORECASE | re.MULTILINE)
    text = re.sub(r"^\s*JUNE\s*\d{1,2},\s*\d{4}\s*NEJM\.ORG\s*\d{4}\s*\n", "", text, flags=re.IGNORECASE | re.MULTILINE)
    text = re.sub(r"^\s*\d+\s*N Engl J Med\s+.*NEJM.ORG.*\n", "", text, flags=re.IGNORECASE | re.MULTILINE)
    text = re.sub(r"^\s*PD-1 BLOCKADE IN MISMATCH-REPAIR DEFICIENCY\s*\n", "", text, flags=re.IGNORECASE | re.MULTILINE)
    text = re.sub(r"^\s*[A-Za-z\s]+ et al\.\s*\n", "", text, flags=re.MULTILINE)

    text = re.sub(r"^\s*\d+\s*\n", "", text, flags=re.MULTILINE)
    text = re.sub(r"\n\s*\d+\s*$", "", text, flags=re.MULTILINE)

    text = re.sub(r"The New England Journal of Medicine is produced by NEJM Group.*\n", "", text, flags=re.IGNORECASE)
    text = re.sub(r"Downloaded from nejm.org on .*\. For personal use only\.\n", "", text, flags=re.IGNORECASE)
    text = re.sub(r"No other uses without permission\. Copyright © \d{4} Massachusetts Medical Society\. All rights reserved\.\n", "", text, flags=re.IGNORECASE)
    text = re.sub(r"Copyright © \d{4} Massachusetts Medical Society\.\n?", "", text, flags=re.IGNORECASE)

    text = re.sub(r"(\w)-(\s*)\n(\s*)(\w)", r"\1\4", text)
    text = re.sub(r"\n\s*\n", "\n\n", text)
    text = re.sub(r" +", " ", text)
    text = text.strip()
    return text


def preprocess_loaded_pages(raw_pages: List[Document]) -> List[Document]:
    """
    로드된 원본 Document 객체 리스트를 받아 각 페이지 내용을 전처리합니다.
    반환값: 전처리된 Document 객체들의 리스트 (페이지 단위)
    """
    if not raw_pages:
        print("No raw pages to preprocess.")
        return []

    all_processed_pages: List[Document] = []
    print("\nPreprocessing loaded pages...")
    for page_doc in raw_pages:
        current_metadata: Dict[str, Any] = page_doc.metadata.copy()
        source_file = current_metadata.get('source', 'Unknown source')
        page_num_raw = current_metadata.get('page', -1) # 0-indexed

        processed_content = preprocess_page_content(
            page_doc.page_content,
            source_file,
            page_num_raw
        )
        
        if processed_content.strip():
            all_processed_pages.append(Document(page_content=processed_content, metadata=current_metadata))
        else:
            page_num_display = page_num_raw + 1 if page_num_raw != -1 else "Unknown"
            print(f" Page {page_num_display} in {os.path.basename(source_file)} resulted in empty content after preprocessing.")
            
    print(f"Total pages after preprocessing (and potential empty page removal): {len(all_processed_pages)}")
    return all_processed_pages

def split_text_documents(documents: List[Document], chunk_size: int = 1500, chunk_overlap: int = 200) -> List[Document]:
    """
    Document 객체 리스트를 받아서 텍스트를 청크 단위로 분할합니다.
    반환값: 분할된 Document 객체들의 리스트 (청크 단위)
    """
    if not documents:
        print("No documents to split.")
        return []

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=[
            "\n\n",  # 1. 문단
            "\n",    # 2. 줄바꿈
            ". ",    # 3. 문장 (마침표 뒤 공백)
            "? ",    # 4. 문장 (물음표 뒤 공백)
            "! ",    # 5. 문장 (느낌표 뒤 공백)
            "Fig.",
            "Table ",
            " ",     # 6. 단어
            "",      # 7. 문자
        ],
        length_function=len,
        is_separator_regex=False,
    )

    print(f"\nSplitting {len(documents)} processed documents into chunks...")
    split_docs = text_splitter.split_documents(documents)
    print(f"Documents split into {len(split_docs)} chunks.")
    return split_docs

In [26]:
paper_directory = "../paper"  

# 디렉토리 존재 확인
if not os.path.exists(paper_directory):
    os.makedirs(paper_directory)
    print(f"Directory '{paper_directory}' was created. Please add PDF files to test.")

# 1. PDF에서 원본 페이지 로드
raw_document_pages = load_raw_papers_from_pdfs(paper_directory)

Loading (raw): Chimeric Antigen Receptor T Cells.pdf
Loading (raw): Immuno-oncology upset in bladder cancer.pdf
Loading (raw): Osimertinib in Untreated EGFR-Mutated Advanced.pdf
Loading (raw): PD-1 Blockade in Tumors.pdf

Total raw pages loaded: 37


In [27]:
# 2. 로드된 페이지들 전처리
processed_document_pages = preprocess_loaded_pages(raw_document_pages)


Preprocessing loaded pages...
Total pages after preprocessing (and potential empty page removal): 37


In [29]:
# 3. 전처리된 페이지들을 청크로 분할
split_chunks = split_text_documents(processed_document_pages, chunk_size=1000, chunk_overlap=100)


Splitting 37 processed documents into chunks...
Documents split into 199 chunks.


In [30]:
print(f"\n--- 최종 분할 결과 요약 ---")
print(f"총 분할된 청크 개수: {len(split_chunks)}")
print("-" * 30)

# 처음 2개 청크 미리보기
for i, doc_chunk in enumerate(split_chunks[:2]):
    print(f"청크 {i+1} 메타데이터: {doc_chunk.metadata}")
    print(f"청크 {i+1} 미리보기 (첫 300자):\n{doc_chunk.page_content[:300]}...\n")
    print("-" * 30)

# 마지막 청크 미리보기 (존재한다면)
if len(split_chunks) > 2:
    print(f"마지막 청크 메타데이터: {split_chunks[-1].metadata}")
    print(f"마지막 청크 미리보기 (첫 300자):\n{split_chunks[-1].page_content[:300]}...\n")
    print("-" * 30)



--- 최종 분할 결과 요약 ---
총 분할된 청크 개수: 199
------------------------------
청크 1 메타데이터: {'producer': 'Adobe PDF Library 10.0.1; modified using iText 4.2.0 by 1T3XT', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'creationdate': '2016-02-18T15:38:50-05:00', 'moddate': '2025-05-28T04:37:59-07:00', 'trapped': '/False', 'subject': 'N Engl J Med 2014.371:1507-1517', 'title': 'Chimeric Antigen Receptor T Cells for Sustained Remissions in Leukemia', 'source': '../paper\\Chimeric Antigen Receptor T Cells.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1507'}
청크 1 미리보기 (첫 300자):
The new england journal of medicine
n engl j med 371;16 nejm.org October 16, 2014 1507
From the Division of Oncology, Chil -
dren’s Hospital of Philadelphia (S.L.M., 
R.A., D.M.B., N.J.B., S.R.R., D.T.T., S.A.G.), 
the Departments of Pediatrics (S.L.M., 
R.A., D.M.B., N.J.B., S.R.R., D.T.T., S.A.G.)...

------------------------------
청크 2 메타데이터: {'producer': 'Adobe PDF Library 10.0.1; modified using iText 4.2.0 by 1T3XT', 'c